## Задание


- Объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
- Провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
- Сделать визуализацию кластеров тематик
- Проинтерпритировать получившиеся тематики

---

In [1]:
!pip install pymorphy2
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 1.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached future-0.18.2-py3-none-any.whl
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=cc740a88fe072b7f948c02a20db295522e16b4bcb23b17e0ebc9f5ea39c1bafb
  Stored in directory: /Users/ilya.ivolgin/Library/Caches/pip/wheels/57/a4/86/d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=4c0f5e391f9df4ce7cda7745aa6cd4e6a335a52eff5491b90c067d4ebbe7dded
  Stored in directory: /Users/ilya.ivolgin/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built pyLDAvis sklearn


In [5]:
!pip install pymystem3

---

In [6]:
# Import libraries

import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
import pymorphy2
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilya.ivolgin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


---

In [7]:
positive = pd.read_csv('positive.csv', sep = ';', usecols = [3], names = ['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index = True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [9]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4] 

morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian') 

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
/var/folders/wf/0ws1q6ks6yv_xwm2_24s254h0000gn/T/ipykernel_96185/747707785.py:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is depreca

In [10]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


---

In [11]:
dictionary = corpora.Dictionary(df['text']) 
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n = None) 
dictionary.save('tweet.dict')

len(dictionary.keys())

8172

In [12]:
corpus = [dictionary.doc2bow(text) for text in df['text']] 
corpora.MmCorpus.serialize('tweet.model', corpus)

In [13]:
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


In [14]:
lda = ldamodel.LdaModel(corpus, id2word = dictionary, num_topics = 3)
lda.show_topics(num_topics = 3, num_words = 20, formatted = True)

[(0,
  '0.015*"самый" + 0.010*"человек" + 0.009*"скоро" + 0.009*"найти" + 0.009*"экзамен" + 0.008*"дело" + 0.008*"казаться" + 0.008*"хороший" + 0.008*"прийтись" + 0.008*"нравиться" + 0.008*"наш" + 0.007*"правда" + 0.006*"ранний" + 0.006*"нужный" + 0.006*"парень" + 0.006*"чувство" + 0.006*"город" + 0.006*"конец" + 0.005*"момент" + 0.005*"долго"'),
 (1,
  '0.026*"завтра" + 0.024*"день" + 0.018*"дом" + 0.017*"школа" + 0.015*"сегодня" + 0.013*"жизнь" + 0.011*"жаль" + 0.010*"час" + 0.010*"новый" + 0.010*"время" + 0.009*"друг" + 0.009*"последний" + 0.009*"год" + 0.009*"урок" + 0.009*"идти" + 0.008*"ночь" + 0.008*"первый" + 0.008*"мама" + 0.008*"неделя" + 0.007*"работа"'),
 (2,
  '0.022*"сегодня" + 0.022*"больш" + 0.018*"человек" + 0.015*"утро" + 0.013*"настроение" + 0.009*"хороший" + 0.009*"день" + 0.009*"улица" + 0.008*"снег" + 0.007*"боль" + 0.007*"твиттер" + 0.007*"спасибо" + 0.007*"минута" + 0.007*"слово" + 0.007*"вечер" + 0.007*"ужасный" + 0.007*"закончиться" + 0.006*"деньга" + 0.006*"б

In [15]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative